# Task 1 : Data Collection

In [357]:
import requests
import shutil
from datetime import date,timedelta
import calendar
from zipfile import ZipFile
import pandas as pd
from pathlib import Path 
import urllib.request

In [381]:
DATASET_PATH = Path("dataset")
DATASET_CLEANED_PATH = Path("dataset_cleaned")
CHROME_DRIVER_PATH=Path("chrome_driver")

In [323]:
if os.path.exists(DATASET_PATH) and os.path.isdir(DATASET_PATH):
    shutil.rmtree(DATASET_PATH)
if os.path.exists(DATASET_CLEANED_PATH) and os.path.isdir(DATASET_CLEANED_PATH):
    shutil.rmtree(DATASET_CLEANED_PATH)

In [324]:
DATASET_PATH.mkdir(parents=True, exist_ok=True)
DATASET_CLEANED_PATH.mkdir(parents=True, exist_ok=True)
CHROME_DRIVER_PATH.mkdir(parents=True, exist_ok=True)

In [325]:
# https://archives.nseindia.com/content/historical/EQUITIES/2023/SEP/cm21SEP2023bhav.csv.zip
# https://archives.nseindia.com/content/historical/EQUITIES/2023/SEP/cm22SEP2023bhav.csv.zip
# https://strftime.org/

def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        print("Inside the download_file function")
        DATASET_FILE_NAME=DATASET_PATH / local_filename
        with open(DATASET_FILE_NAME, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

In [326]:
number_of_days=5
base_url="https://archives.nseindia.com/content/historical/EQUITIES/2023/#/cm{}bhav.csv.zip"

#current_date=date.today()
#reference_date=date(2023,11,3)
#reference_date=date(2023,10,27) #Oct 24,2023 is holiday
reference_date=date(2023,11,3)


i=0 # to check number of successful downloads which shall be equal of number of dasy after successfully download
j=0 # to check number of days skipped

In [382]:
from bs4 import BeautifulSoup
from selenium import webdriver    

options = webdriver.ChromeOptions()
options.add_argument('--headless')
# executable_path param is not needed if you updated PATH
browser = webdriver.Chrome(options=options, executable_path=CHROME_DRIVER_PATH/'chromedriver.exe')
browser.get("https://www.nseindia.com/resources/exchange-communication-holidays")
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")
print(soup)
browser.quit()

C:\Users\2379\AppData\Local\Temp/ipykernel_2100/2504275503.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path=CHROME_DRIVER_PATH/'chromedriver.exe')


WebDriverException: Message: <HTML><TITLE>Forbidden</TITLE><H1>Self-referential requests to proxy are forbidden</H1>Self-referential requests to this proxy server are forbidden</HTML>



In [379]:
import requests
URL = 'https://www.nseindia.com/resources/exchange-communication-holidays#holiday_trading'
#URL = 'https://www.google.co.in'
headers_default = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
resp = requests.get(URL,headers=headers_default,timeout=(3.05, 27))
print(resp.text)
#df  = pd.read_html(resp.text)

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" />
<title>
    Market Timings &amp; Holidays - NSE India
</title>
<meta name="description" content="Market Timings &amp; Holidays: Get latest information about Market Timings &amp; Holidays and more! Click here to stay updated and invest wisely! Learn more about Market Timings &amp; Holidays Today, visit NSE India." />
<meta name="keywords" content="" />
<meta name="robots" content="index,follow" />
<link rel="shortcut icon" href="/assets/images/favicon.ico" type="image/x-icon" />
<meta property="og:title" content="" />
<meta property="og:description" content="" />
<meta property="og:image" content="" />
<meta property="og:url" cont

In [358]:
# retrieve holidays list from NSE page 'https://www.nseindia.com/resources/exchange-communication-holidays'#
df_holidays=pd.read_html("https://www.nseindia.com/resources/exchange-communication-holidays")

RemoteDisconnected: Remote end closed connection without response

In [328]:
temp=reference_date

while i<number_of_days:  
    if calendar.day_name[temp.weekday()] in 'SaturdaySunday':        
        print(" {} is weekend".format(temp.strftime("%d%b%Y").upper()))
    else:
        url_for_csv=base_url.replace('{}',temp.strftime("%d%b%Y").upper())
        url_for_csv=url_for_csv.replace('#',temp.strftime("%b").upper())
        print(url_for_csv)
        if (urllib.request.urlopen(url=url_for_csv,timeout=15).getcode()) == 200:
            download_file(url_for_csv)
            i=i+1    
            print("Downloaded file for  {}".format(temp.strftime("%d%b%Y").upper()))
            DATASET_FILE_NAME=DATASET_PATH / url_for_csv.split('/')[-1]
            with ZipFile(DATASET_FILE_NAME, 'r') as f:
                #extract in current directory
                f.extractall(DATASET_PATH)
        temp=temp+timedelta(days=-1)
   

https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm03NOV2023bhav.csv.zip
Inside the download_file function
Downloaded file for  03NOV2023
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm02NOV2023bhav.csv.zip
Inside the download_file function
Downloaded file for  02NOV2023
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm01NOV2023bhav.csv.zip
Inside the download_file function
Downloaded file for  01NOV2023
https://archives.nseindia.com/content/historical/EQUITIES/2023/OCT/cm31OCT2023bhav.csv.zip
Inside the download_file function
Downloaded file for  31OCT2023
https://archives.nseindia.com/content/historical/EQUITIES/2023/OCT/cm30OCT2023bhav.csv.zip
Inside the download_file function
Downloaded file for  30OCT2023


# Task 2 : Data Cleaning

In [329]:
import pandas as pd 
import os 
import glob 
from datetime import datetime

In [330]:
# use glob to get all the csv files  
# in the folder 
#path = os.getcwd() 
csv_files = glob.glob(os.path.join(DATASET_PATH, "*.csv")) 

In [331]:
  
# loop over the list of csv files 
for f in csv_files: 
      
    # read the csv file 
    df = pd.read_csv(f) 
      
    # print the location and filename 
    print('Location:', f) 
    file_name= f.split("\\")[-1]
    print('File Name:', file_name) 

    sub1 = "cm"
    sub2 = "bhav"
 
    file_name=file_name.replace(sub1,"*")
    file_name=file_name.replace(sub2,"*")
    re=file_name.split("*")
    res=re[1]

    date_str=res
    date_format = '%d%b%Y'
    date_obj = datetime.strptime(date_str, date_format)

    # filter EQ values
    df=df[df['SERIES'].isin(['EQ'])]
    file_name=date_obj.strftime("%Y%m%d")+".csv"
    DATASET_CLEANED_FILE_NAME=DATASET_CLEANED_PATH / file_name
    df=df.dropna(how='all', axis='columns')
    df.to_csv(DATASET_CLEANED_FILE_NAME,index=False)

Location: dataset\cm01NOV2023bhav.csv
File Name: cm01NOV2023bhav.csv
Location: dataset\cm02NOV2023bhav.csv
File Name: cm02NOV2023bhav.csv
Location: dataset\cm03NOV2023bhav.csv
File Name: cm03NOV2023bhav.csv
Location: dataset\cm30OCT2023bhav.csv
File Name: cm30OCT2023bhav.csv
Location: dataset\cm31OCT2023bhav.csv
File Name: cm31OCT2023bhav.csv


# Task 3 : Data Storage

In [332]:
! pip install mysql-connector-python


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [333]:
! pip install SQLAlchemy


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [334]:
! pip install mysqlclient


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [335]:
DB_SCRIPTS_PATH=Path("DB_Scripts")
DB_SCRIPTS_PATH.mkdir(parents=True, exist_ok=True)
DB_CREATION_FILE_PATH=DB_SCRIPTS_PATH / "create_analytics_database.sql"

In [336]:
import mysql.connector 

In [337]:
# function to execute sql file
def execute_sql_file(DB_CREATION_FILE_PATH):
    # establish a connection to the database 
    cnx = mysql.connector.connect(user='root', password='root',host='127.0.0.1')# database='your_database')
    
    #cnx = create_engine("mysql+mysqldb://{user}:{pw}@{host}/{db}".format(host="localhost", db="analytics", user="root", pw="root"))

    # create a cursor object 
    cursor = cnx.cursor() 

    # read the contents of the .sql file 
    with open(DB_CREATION_FILE_PATH, 'r') as file: 
        sql_script = file.read() 

    # execute the script using the cursor object 
    for result in cursor.execute(sql_script, multi=True): 
        pass 

    # commit the changes to the database 
    cnx.commit() 

    # close the cursor and connection 
    cursor.close() 
    cnx.close() 

In [338]:
# Creating a database if it doesnt exist
execute_sql_file(DB_CREATION_FILE_PATH)

In [339]:
TABLE_CREATION_FILE_PATH=DB_SCRIPTS_PATH / "create_eq_table.sql"

In [340]:
# Create table schema if it doesnt exist
execute_sql_file(TABLE_CREATION_FILE_PATH)

In [341]:
csv_files = glob.glob(os.path.join(DATASET_CLEANED_PATH, "*.csv")) 

In [342]:
from sqlalchemy import create_engine
# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+mysqldb://{user}:{pw}@{host}/{db}".format(host="localhost", db="analytics", user="root", pw="root"))

In [343]:
#cnx = mysql.connector.connect(user='root', password='root',host='127.0.0.1')# database='your_database') 
# loop over the list of csv files
df=[]
for f in csv_files: 
    # read the csv file 
    df.append(pd.read_csv(f))
    
df_result=pd.concat(df,axis=1)
    
df_result.to_sql(con=engine, name='tbl_eq_data', if_exists='replace')

1771

# Task 4 : Main Process

In [344]:
TABLE_CREATION_FILE_PATH=DB_SCRIPTS_PATH / "create_eq_analytics_table.sql"

In [345]:
# Create table schema if it doesnt exist
execute_sql_file(TABLE_CREATION_FILE_PATH)

In [346]:
df=[]
for f in csv_files: 
    # read the csv file 
    df.append(pd.read_csv(f))
    
df_result=pd.concat(df,axis=0)
    
df_result=df_result[['SYMBOL','TIMESTAMP','CLOSE','TOTTRDQTY']]

In [347]:
df_result.head()

,SYMBOL,TIMESTAMP,CLOSE,TOTTRDQTY
0,21STCENMGM,30-OCT-2023,18.25,2663
1,360ONE,30-OCT-2023,518.10,174556
2,3IINFOLTD,30-OCT-2023,36.90,400942
3,3MINDIA,30-OCT-2023,29597.60,805
4,3PLAND,30-OCT-2023,26.75,16132


In [348]:
#sort data date wise
df_result=df_result.sort_values(by=['TIMESTAMP','SYMBOL'],ascending=[False,True])

In [349]:
df_result.shape

(8847, 4)

In [350]:
# calculate moving average for the last 2 days
df_result['moving_average']=df_result['CLOSE'].rolling(2).mean()

In [351]:
df_result.to_sql(con=engine, name='tbl_eq_analytics', if_exists='replace')

8847

# Task 5 : Output

In [353]:
df_result_2=df_result[df_result['CLOSE'] > df_result['moving_average']]

In [354]:
df_result_2.head()

,SYMBOL,TIMESTAMP,CLOSE,TOTTRDQTY,moving_average
1,360ONE,31-OCT-2023,525.10,369884,271.600
3,3MINDIA,31-OCT-2023,29762.70,1045,14899.550
5,5PAISA,31-OCT-2023,444.50,94823,235.450
9,AARON,31-OCT-2023,294.30,5568,150.425
10,AARTIDRUGS,31-OCT-2023,469.45,125225,381.875


In [355]:
OUTPUT_PATH = Path("output")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_FILE_NAME=OUTPUT_PATH/ "output.csv"

In [356]:
df_result_2.to_csv(OUTPUT_FILE_NAME,index=False)